# Movie Recommendation Model

In [1]:
import pandas as pd
import numpy as np

import gzip
import json

%load_ext autoreload
%autoreload 2

## Import Data

In [2]:
# !wget -P ../../data/meta_Movies_and_TV.json.gz http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Movies_and_TV.json.gz
# !wget -P ../../data/Movies_and_TV.json.gz http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Movies_and_TV.json.gz
# !gzip -d  ../../data/meta_Movies_and_TV.json.gz
#  !gzip -d  ../../data/Movies_and_TV.json.gz
# !mv meta_Movies_and_TV.json ../../data
# !mv mMovies_and_TV.json ../../data

--2021-04-29 16:47:21--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Movies_and_TV.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104130019 (99M) [application/octet-stream]
Saving to: ‘../../data/meta_Movies_and_TV.json.gz/meta_Movies_and_TV.json.gz.2’

meta_Movies_and_TV. 100%[===================>]  99.31M  11.0MB/s    in 9.4s    

2021-04-29 16:47:30 (10.6 MB/s) - ‘../../data/meta_Movies_and_TV.json.gz/meta_Movies_and_TV.json.gz.2’ saved [104130019/104130019]

--2021-04-29 16:47:31--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Movies_and_TV.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1538847239 (1.4G) [application/octet-stream]
Saving to

## Read Metafile 

In [3]:
# open the json.gz file for meta dataset
data_meta = []
with gzip.open('../../data/meta_Movies_and_TV.json.gz/meta_Movies_and_TV.json.gz') as f:
    for line in f:
        data_meta.append(json.loads(line.strip()))
    
# total length of list, this number equals total number of products
print(len(data_meta))

# first row of the list
print(data_meta[0])

203766
{'category': ['Movies & TV', 'Movies'], 'tech1': '', 'description': [], 'fit': '', 'title': 'Understanding Seizures and Epilepsy', 'also_buy': [], 'image': [], 'tech2': '', 'brand': '', 'feature': [], 'rank': '886,503 in Movies & TV (', 'also_view': [], 'main_cat': 'Movies & TV', 'similar_item': '', 'date': '', 'price': '', 'asin': '0000695009'}


## Data Preparation

In [49]:
# open the json.gz file for meta dataset
mtv_data = []
with gzip.open(f'../../data/Movies_and_TV.json.gz/Movies_and_TV.json.gz') as f:
    for line in f:
        df = json.loads(line.strip())
        if (df['reviewTime'].endswith('2016') |
           df['reviewTime'].endswith('2017') |
            df['reviewTime'].endswith('2018')):
            mtv_data.append(df)
        else:
            continue
    # Number of records selected
    print(len(mtv_data))

2841447


#### Create dataframes from the data just read

In [50]:
# Create a dataframe from the dictionary
df_mtv = pd.DataFrame(mtv_data)
df_mtv = df_mtv.drop('image', axis=1)
df_mtv = df_mtv[df_mtv['verified']==True]

In [51]:
# convert data_meta to df
df_meta = pd.DataFrame.from_dict(data_meta)

# drop useless columns
df_meta = df_meta.dropna(subset=['details'])
df_meta = df_meta.drop(['image', 'feature', 'date', 'tech1'], axis=1)

In [52]:
df_mtv_meta = df_mtv.merge(df_meta, on='asin', how='inner')

## Create a csv file for further exploration.

Because the data file is fairly large, I am storing the selected fields in a csv file.

In [53]:
df_mtv_meta.to_csv('../../data/mtv_meta.csv', index=False)